<sup>This notebook is originally from https://github.com/lizanafj/UHI_mapping and licensed for reuse under [GPL-3.0 license].</sup>

# Data Analytics - Annual Diagnosis 
---

This notebook has different data analysis techniques for the diagnosis of the urban heat island in cities

In [ ]:
import os
import requests
import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt #biblioteca de generación de gráficos
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from matplotlib.dates import date2num
import datetime
from datetime import datetime

# get folder location of script
cwd = os.getcwd()

## Define name of city project to work
---

Define the name of the **`city`** (projectname) to upload/save all data during the workflow

In [ ]:
#City project: 
city= "London"

## Import input data of the project - config.py 
---

In [ ]:
os.chdir(cwd)

In [ ]:
cwd_main = cwd[:-10]
cwd_project = cwd_main + f"\projectname\{city}"
print(cwd_main)
print(cwd_project)

In [ ]:
#import config.py file

# go to current location of script
os.chdir(cwd_project)
import config as cf

In [ ]:
#check input variables
print("city: ",cf.city)
print("first date: ",cf.first_date)
print("last date: ",cf.last_date)
print("lat,long: ",cf.lat,cf.long)
print("plot: ",cf.plot)
print("extent:",cf.extent)

## Data analytics - Annual Diagnosis
---

Read final files after pre-processing and preliminary analytical steps

In [ ]:
#Read data from OWS
os.chdir(cf.cwd_data_filt)
os.listdir()

In [ ]:
coord_OWS = pd.read_csv("Coordinates_London_2021_OWS_QC_I5.csv")

In [ ]:
#Read data from CWS
os.chdir(cf.cwd_data_proc)
os.listdir()

In [ ]:
coord_CWS = pd.read_csv("Coordinates_London_2021_CWS_all_QC_G8_proc_CDH.csv")

In [ ]:
coord_all = pd.concat([coord_OWS,coord_CWS]).reset_index().drop(columns=["Unnamed: 0.1","index","Unnamed: 0"])

In [ ]:
coord_all

Mapping weather stations over streemap - using Cartopy & street map API

In [ ]:
extent = cf.extent

In [ ]:
#Cartopy
import cartopy 
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#kriging
import pykrige #biblioteca de geoestadística - https://geostat-framework.readthedocs.io/projects/pykrige/en/stable/
import pykrige.kriging_tools as kt #herramientas para exportar e importar archivos .asc con biblioteca PyKrige
from pykrige.ok import OrdinaryKriging #Kriging Ordinario con PyKrige

from matplotlib.cm import ScalarMappable

#upload functions
os.chdir(cf.cwd_scripts_visual)
from functions.streetmap import image_spoof #importamos street map API

In [ ]:
#Visualisation of location of weather stations
x = np.array(coord_all['long'])
y = np.array(coord_all['lat'])
temp=np.array(coord_all['type'])

cimgt.Stamen.get_image = image_spoof # reformat web request for street map spoofing
osm_img = cimgt.Stamen('terrain-background') # spoofed, downloaded street map  'terrain-background', 'terrain'

fig = plt.figure(figsize=(12,9)) # open matplotlib figure
ax1 = plt.axes(projection=osm_img.crs) # project using coordinate reference system (CRS) of street map
ax1.set_title('Location of weather stations',fontsize=16)

ax1.set_extent(extent) # set extents
ax1.set_xticks(np.linspace(extent[0],extent[1],7),crs=ccrs.PlateCarree()) # set longitude indicators
ax1.set_yticks(np.linspace(extent[2],extent[3],7)[1:],crs=ccrs.PlateCarree()) # set latitude indicators
lon_formatter = LongitudeFormatter(number_format='0.1f',degree_symbol='',dateline_direction_label=True) # format lons
lat_formatter = LatitudeFormatter(number_format='0.1f',degree_symbol='') # format lats
ax1.xaxis.set_major_formatter(lon_formatter) # set lons
ax1.yaxis.set_major_formatter(lat_formatter) # set lats
ax1.xaxis.set_tick_params(labelsize=14)
ax1.yaxis.set_tick_params(labelsize=14)

scale = np.ceil(-np.sqrt(2)*np.log(np.divide((extent[1]-extent[0])/2.0,350.0))) # empirical solve for scale based on zoom
scale = (scale<20) and scale or 19 # scale cannot be larger than 19
ax1.add_image(osm_img, int(scale)) # add OSM with zoom specification

#visualisation of stations as scatters/dots
Color_list = ["red","orange","k"]
marker_size1 = [4,4,8]
marker1= ['D','D','o']
uniq = sorted(list(set(coord_all['type'])),reverse = False)

# Plot each species
for i in range(len(uniq)):
    indx = coord_all['type'] == uniq[i]
    ax1.plot(x[indx], y[indx], markersize=marker_size1[i],marker=marker1[i],linestyle='',color=Color_list[i],label=uniq[i],transform=ccrs.PlateCarree())

plt.legend(loc='lower right',fontsize=14)

os.chdir(cf.cwd_figures)
plt.savefig(f"05_Analytics_{city}_figure1.jpg", format='jpg')
plt.show()

Annual CDHs mapping by kriging interpolation over streetmap

In [ ]:
##############################################################################  
# ANNUAL CDH INTERPOLATION USING KRIGING
##############################################################################

x = np.array(coord_CWS['long'])
y = np.array(coord_CWS['lat'])
temp=np.array(coord_CWS['cdh_total']) #'cdh_night' , 'cdh_day', 'cdh_total' 

###############################################################################
# Create the ordinary kriging object. 

OK = OrdinaryKriging(x,y,temp, variogram_model="spherical", nlags=15,weight= True,  
                      enable_plotting=True, 
                    coordinates_type='geographic',enable_statistics=True)

grid_x = np.arange(extent[0],extent[1],0.005) #size of pixel ########## IMPORTANT 0.005 defaout
grid_y = np.arange(extent[2],extent[3],0.005)

###############################################################################
# Creates the kriged grid and the variance grid. Allows for kriging on a rectangular
# grid of points, on a masked rectangular grid of points, or with arbitrary points.
z, ss = OK.execute('grid', grid_x, grid_y)

#save results of kriging
# Writes the kriged grid to an ASCII grid file and plot it.
os.chdir(cf.cwd_figures)
kt.write_asc_grid(grid_x, grid_y, z, filename="kriging_ordinario.asc") #Kriging Ordinario en forma de matriz
kt.write_asc_grid(grid_x, grid_y, ss, filename="kriging_ordinario_var.asc") #Varianza de estimación en forma de matriz

#-------------------------------------------------------------------------------------------

In [ ]:
#upload results of kriging
os.chdir(cf.cwd_figures)
asc = pd.read_csv("./kriging_ordinario.asc", header=None, skiprows=7, sep="\s+") 
temp2=np.array(asc)

##############################################################################
#PLOT kriging over map using CARTOPY - street map API
#-------------------------------------------------------------------------------------------

cimgt.Stamen.get_image = image_spoof # reformat web request for street map spoofing
osm_img = cimgt.OSM() # spoofed, downloaded street map  'terrain-background', 'terrain'  'toner'

fig = plt.figure(figsize=(12,9)) # open matplotlib figure
ax1 = plt.axes(projection=osm_img.crs) # project using coordinate reference system (CRS) of street map
ax1.set_title('UHI Intensity. Total CDHs ',fontsize=16)

ax1.set_extent(extent) # set extents
ax1.set_xticks(np.linspace(extent[0],extent[1],7),crs=ccrs.PlateCarree()) # set longitude indicators
ax1.set_yticks(np.linspace(extent[2],extent[3],7)[1:],crs=ccrs.PlateCarree()) # set latitude indicators
lon_formatter = LongitudeFormatter(number_format='0.1f',degree_symbol='',dateline_direction_label=True) # format lons
lat_formatter = LatitudeFormatter(number_format='0.1f',degree_symbol='') # format lats
ax1.xaxis.set_major_formatter(lon_formatter) # set lons
ax1.yaxis.set_major_formatter(lat_formatter) # set lats
ax1.xaxis.set_tick_params(labelsize=14)
ax1.yaxis.set_tick_params(labelsize=14)

scale = np.ceil(-np.sqrt(2)*np.log(np.divide((extent[1]-extent[0])/2.0,350.0))) # empirical solve for scale based on zoom
scale = (scale<20) and scale or 19 # scale cannot be larger than 19
ax1.add_image(osm_img, int(scale)) # add OSM with zoom specification

cmap = plt.get_cmap("turbo") #Color alternatives: "turbo" "Reds" "hot_r" "jet"   -   "OrRd" "gist_heat_r" "afmhot_r" "RdBu_r"
ax1=plt.imshow(temp2, transform=ccrs.PlateCarree(),cmap=cmap,extent=[extent[0],extent[1],extent[2],extent[3]],alpha=0.65,zorder=1,vmin=temp2.min(), vmax=temp2.max()) #min and max similar to bar

#temperature bar
norm = plt.Normalize(temp2.min(),temp2.max()) #max and min   -  temp2.min(),temp2.max()
sm =  ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])
cbaxes = fig.add_axes([0.73, 0.15, 0.025, 0.3])
cbar = fig.colorbar(sm, cax=cbaxes, orientation='vertical',shrink=0.3,aspect=10)
cbar.ax.set_title("CDH")

os.chdir(cf.cwd_figures)
plt.savefig(f"05_Analytics_{city}_figure2.jpg", format='jpg')
plt.show()